# Emission model basics

Here we give an overview of the basic functionality of an ``EmissionModel``. Elsewhere we cover [what models are available](premade_models/premade_models.rst), [how to modify a model](modify_models.ipynb), [making your own custom models](custom_models.ipynb), and [combining models from different components](combined_models.ipynb).

## Operations
 
To use an ``EmissionModel`` model we simply instantiate it with the required arguments.
However, which arguments are required depends on the exact operation.  

### Extraction
For *extraction* operations you need:

- A grid to extract from (``grid``).
- A key to extract (``extract``). 

### Combination
For *combination* operations you need:

- A list of models which will be combined (i.e. added) to give the resultant emission (``combine``).

### Generation
For *generation* operations you need:

- A generator class (e.g. a [dust emission model](dust_emission.ipynb)) from which to generate spectra (``generator``). 

### Transformation
For *transformation* operations you need:

- The transformer (e.g. a [dust curve](../attenuation/attenuation.rst) or [igm](../attenuation/igm.ipynb)) to apply (``transformer``/``dust_curve``/``igm``, arguments are interchangable).
- The model to apply the transformation to (``apply_to``).

## Masking
Masking can be applied alongside any of the above operations.
Any number of masks can be combined on the same operation.
Each mask is defined by:

- The attribute of the component to mask on (``mask_attr``).
- The operator to use when generating the mask, i.e. ``"<"``, ``">"``, ``"<="``, ``">="``, ``"=="``, or ``"!="`` (``mask_op``).
- The threshold of the mask (``mask_thresh``).




## Getting a model

For this demonstration we'll load the ``TotalEmission`` premade stellar emisison model.
When using a premade model we need to define certain key parameters here we need a ``dust_curve``, for more details see the [premade model docs](premade_models/premade_models.rst).

In [ ]:
from unyt import Myr

from synthesizer.emission_models import TotalEmission
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.grid import Grid

# Get the grid which we'll need for extraction
grid_dir = "../../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

total = TotalEmission(grid=grid, dust_curve=PowerLaw(slope=-1))

> When using more complex [premade models](premade_models/premade_models.rst), with a deeper "tree", more parameters will be required to populate their "child" models deeper in the tree.
> This tree terminology will become clear later -- don't worry about the specifics here if you are just starting with synthesizer ``EmissionModels``.

In addition to the arguments specfic to each type of model, any model can be passed arguments to define a mask:
- ``mask_attr``, the emitter attribute to define a mask with
- ``mask_op``, the operator to use in the mask
- ``mask_thresh``, the threshold for the mask

In the example below we apply a mask to the stellar ages, so that only emission from stars less than 10 Myr old are returned.

In [ ]:
masked_total = TotalEmission(
    grid=grid,
    label="young_total",
    dust_curve=PowerLaw(slope=-1),
    mask_attr="ages",
    mask_op="<",
    mask_thresh=10 * Myr,
)

> In the above case we used a global mask on the ``TotalEmission`` model, so that every child model in the tree has also been masked.
> In most (but certainly not all) cases you'll only want specific models masked. 
> This can either be done by [constructing your own models](custom_models.ipynb) or [modifying existing ones](modify_models.ipynb). 

## Printing a summary of an ``EmissionModel``

If we want to see a summary of all the models contained within an instance of an ``EmissionModel`` we simply print the model.

In [ ]:
print(masked_total)

This returns a text block describing each model, including its label (in all caps), its emitter (in brackets after the label), and the operation the model performs, followed by the parameters defining the operation.

## Parameter overiding 

Any parameters that aren't listed in the operations above are assumed to be attributes of the "emitter" being used to generate the emission, i.e. a ``Galaxy``, ``Stars``, or ``BlackHole/s``. 
For instance, an extraction can include an escape fraction (``fesc``) or a transformation applying a dust curve will require a V-band optical depth (``tau_v``). Both of these parameters will be extracted from the emitter during the emission calculation. 

However, these parameters can be overidden by passing them as keyword arguments when instantiating the ``EmissionModel``. Any parameters passed in this way will take precedence over the emitter's attributes. 

In [ ]:
fixed_tau_v = TotalEmission(
    grid=grid,
    label="young_total",
    dust_curve=PowerLaw(slope=-1),
    tau_v=0.67,
)
print(fixed_tau_v)

Notice how every model now has tau_v set to 0.67 in the `Fixed parameters` section. 


## Plotting an ``EmissionModel``

For a more detailed view of how models relate to each other we can visualise the tree itself:

In [ ]:
total.plot_emission_tree()
masked_total.plot_emission_tree()

In the tree we can see each individual spectra generated by the model, and how they relate:

- Solid lines denote combinations of spectra. 
- Dashed lines denote a transformation of the child spectra to produce the parent. 
- Dotted lines denote a relationship between the parent generator and it's child spectra used to scale the generated spectra (not required for all generators). 
- Solid outlines have no mask applied. 
- Dashed outlines have a mask applied to them. 
- Square boxes are an extraction operation. 
- Boxes with rounded corners are generation, transformation or combination operations. 

You'll notice the extraction operations are always leaves in the tree.

## Extracting models

An emission model can be treated like a dictionary -- if we want to get a model from somewhere in the tree, we simply index with the model label.

In [ ]:
sub_model = total["reprocessed"]
sub_model.plot_emission_tree()

## Generating spectra and lines

We'll cover this in greater detail in the [spectra docs](../spectra/spectra.rst) and the [lines docs](../lines/lines.rst), but to generate an emission from an ``EmissionModel`` you need to:

- Set up your model as shown in the other ``EmissionModel`` docs.
- Set up an "emitter" (e.g. a component or galaxy containing components) as shown in the [component docs](../components/components.rst).
- Call the emitter's ``get_spectra`` method passing in your model.

This will return the spectra/lines at the root of your emission model and store all generated spectra in the emitter.

> Note that, in addition to fixing parameters as described above, we can also pass any parameter to ``get_spectra`` to override its value on the model/emitter for just that call.

### Generating emissions for each particle

To generate spectra for each particle in a component/galaxy we follow the exact same steps as above, but we need to set ``per_particle=True`` on our model. This can be done at model instantiation...


In [ ]:
# Create a model for per particle emission
total = TotalEmission(
    grid=grid,
    label="total",
    dust_curve=PowerLaw(slope=-1),
    per_particle=True,
)
total.plot_emission_tree()

Or by using the ``set_per_particle`` method on the model. This will set the model it is called from and any child models to generate spectra for each particle in the emitter. Below we'll demonstrate this by making every model in the ``total`` model we made above to be integrated (``per_particle=False``) and then set some of the children to be per particle.

In [ ]:
total.set_per_particle(False)
total["reprocessed"].set_per_particle(True)
total["escaped"].set_per_particle(True)
print(total)
total.plot_emission_tree()

Notice how now the ``"total"`` and ``"attenuated"`` models don't mention per particle emission while all others (which are children of ``"reprocessed"``) have ``Per particle emission: True`` in their summary.

When ``per_particle=True`` the model will generate a spectrum for each particle in the component/galaxy, for each individual model in your ``EmissionModel``, and store these in the ``particle_spectra`` attribute. Note that integrated spectra will automatically be generated too and will be stored in the ``spectra`` attribute.